## Model-Training with pretrained weights from Yolo11n-cls

In [1]:
!nvidia-smi

Thu Dec 11 16:58:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660      WDDM  |   00000000:08:00.0  On |                  N/A |
| 38%   41C    P0             24W /  130W |    1260MiB /   6144MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from ultralytics import YOLO
from pathlib import Path
import os
import shutil

In [3]:
# Paths
DATA_PATH = Path("../data/images_classified")  
PROJECT_NAME = "greenery_classification"       # name of the training run
RUNS_PATH = Path("../runs")                    # output directory

In [ ]:
# Training parameters
MODEL_SIZE = "n" # nano, the smalllest model
EPOCHS = 50
BATCH_SIZE = 32 # can be higher with better GPU
IMAGE_SIZE = 224
PATIENCE = 20  # early stopping patience
AUGMENT = True
DEVICE = "cuda" # using GPU

CLASS_NAMES = ["greenery", "non_greenery"]

In [5]:
# TRAINING

def train_model():
    """Trains the YOLO classification model."""
    print("\n" + "=" * 70)
    print(" STARTING TRAINING")
    print("=" * 70)

    model_path = f"yolo11{MODEL_SIZE}-cls.pt"

    # Check pretrained weights
    weights_dir = Path("./weights")
    weights_dir.mkdir(exist_ok=True)
    weight_file = weights_dir / model_path

    # Load model
    model = YOLO(str(weight_file))

    # Train
    results = model.train(
        data=str(DATA_PATH),
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        patience=PATIENCE,
        project=str(RUNS_PATH),
        name=PROJECT_NAME,
        exist_ok=True,
        plots=True,
        augment=AUGMENT,
        device=DEVICE
    )

    best_model_path = RUNS_PATH / PROJECT_NAME / "weights" / "best.pt"
    print(f"\n Best model saved at: {best_model_path}")

    return model, best_model_path

# VALIDATION + TEST

def evaluate_model(model_path):
    """Runs validation and test."""
    model = YOLO(str(model_path))

    # Validation
    val_metrics = model.val(split="val")
    print(f"\n Validation Top-1 Accuracy: {val_metrics.top1:.2%}")

    # Test
    test_metrics = model.val(split="test")
    print(f"\n Test Top-1 Accuracy: {test_metrics.top1:.2%}")

    return val_metrics, test_metrics

# EXAMPLE PREDICTIONS

def predict_examples(model_path, num_samples=10):
    """Predicts several random samples from the test set."""
    model = YOLO(str(model_path))

    test_images = []
    for cls in CLASS_NAMES:
        cls_images = list((DATA_PATH / "test" / cls).glob("*.jpg"))[:num_samples // 2]
        test_images.extend(cls_images)

    if not test_images:
        print(" No test images found!")
        return

    print(f"\n Predictions for {len(test_images)} sample images:\n")

    for img_path in test_images:
        results = model.predict(img_path, verbose=False)
        probs = results[0].probs

        top1_idx = probs.top1
        top1_conf = probs.top1conf.item()

        predicted_class = CLASS_NAMES[top1_idx]
        true_class = img_path.parent.name

        status = "✅" if predicted_class == true_class else "❌"

        print(
            f"{status} {img_path.name:<30} "
            f"True: {true_class:<13} | Pred: {predicted_class:<13} "
            f"({top1_conf:.2%})"
        )

def main():
    print(" YOLO GREENERY CLASSIFICATION PIPELINE")


    if not DATA_PATH.exists():
        print(f" Dataset not found: {DATA_PATH}")
        return

    # Training
    model, best_model_path = train_model()

    # Evaluation
    val_metrics, test_metrics = evaluate_model(best_model_path)

    # Example Predictions
    predict_examples(best_model_path, num_samples=20)

    print("\n PIPELINE COMPLETE!")
    print(f"All results saved in: {RUNS_PATH / PROJECT_NAME}")


if __name__ == "__main__":
    main()

 YOLO GREENERY CLASSIFICATION PIPELINE

 STARTING TRAINING
Ultralytics 8.3.235  Python-3.11.3 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=..\data\images_classified, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=weights\yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=greenery_classification, nbs=64, nms=False, opset=None, optimiz